In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q htt  https://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [41.5 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hi

https://www.24tutorials.com/spark/flatten-json-spark-dataframe/

In [79]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [80]:
df=spark.read.option("multiline","true").json('/content/Py-SparkcodetoFlattenJSON.json')
df.show()

+--------------------+---------+
|               feeds|totalFeed|
+--------------------+---------+
|[[0, 0, 2020-01-0...|      125|
+--------------------+---------+



In [81]:
df.printSchema()

root
 |-- feeds: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- code: long (nullable = true)
 |    |    |-- commentCount: long (nullable = true)
 |    |    |-- createdAt: string (nullable = true)
 |    |    |-- description: string (nullable = true)
 |    |    |-- feedsComment: string (nullable = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- imagePaths: string (nullable = true)
 |    |    |-- images: string (nullable = true)
 |    |    |-- isdeleted: boolean (nullable = true)
 |    |    |-- lat: long (nullable = true)
 |    |    |-- likeDislike: struct (nullable = true)
 |    |    |    |-- dislikes: long (nullable = true)
 |    |    |    |-- likes: long (nullable = true)
 |    |    |    |-- userAction: long (nullable = true)
 |    |    |-- lng: long (nullable = true)
 |    |    |-- location: string (nullable = true)
 |    |    |-- mediatype: long (nullable = true)
 |    |    |-- msg: string (nullable = true)
 |    |    |-- multiMe

In [82]:
def all_columns_apart_from_the_particular_column(df,name_string):
  ms=df.schema.fields;
  ag=list(filter(lambda x: x.name!=name_string,ms));
  am=list(map(lambda x:x.name,ag));
  return am;

In [83]:
def expand_structure(df,dfstruct,al):
  ag=[];
  ag=list(map(lambda x:al+"."+x.name,dfstruct))
  return ag;

In [84]:
def flat_df(df):
  all_raw_fields=df.schema.fields;
  for i in all_raw_fields:
    if (str(i.dataType)[0:9])=="ArrayType":
      column_list=all_columns_apart_from_the_particular_column(df,i.name);
      explosion="explode("+i.name+")" "as "+"Exploded_"+i.name;
      column_list.append(explosion);
      df=df.selectExpr(column_list);
    elif (str(i.dataType)[0:10])=="StructType":
      column_list=all_columns_apart_from_the_particular_column(df,i.name);
      dfstruct=df.schema[i.name].dataType.fields;
      all_fields_within_structure=expand_structure(df,dfstruct,i.name);
      proper_name = [ x+" as "+ x.replace('.', "_") for x in all_fields_within_structure ];
      column_list.extend(proper_name)
      df=df.selectExpr(column_list);
  return df;

In [85]:
def encapsulation(df):
  ms=str(df.schema);
  if (("ArrayType" in ms)):
    processing=flat_df(df);
    return encapsulation(processing);
  elif (ms.count("StructType")>1):
    processing=flat_df(df);
    return encapsulation(processing);
  else:
    return (df);

In [86]:
df1=encapsulation(df);

In [87]:
df1.show()

+---------+-------------------+---------------------------+------------------------+--------------------------+---------------------------+-----------------+-------------------------+---------------------+------------------------+------------------+------------------+-----------------------+------------------------+------------------+-------------------+-----------------------------+--------------------+---------------------+-----------------------+-----------------------------------+--------------------------------+-------------------------------------+-------------------------------------------+----------------------------------------------+-------------------------------------+--------------------------------------------+--------------------------------------------+---------------------------------------+----------------------------------------+--------------------------------------+
|totalFeed|Exploded_feeds_code|Exploded_feeds_commentCount|Exploded_feeds_createdAt|Exploded_feeds_d

In [88]:
df1.count(),len(df1.columns)

(296, 31)

In [89]:
df1.printSchema()

root
 |-- totalFeed: long (nullable = true)
 |-- Exploded_feeds_code: long (nullable = true)
 |-- Exploded_feeds_commentCount: long (nullable = true)
 |-- Exploded_feeds_createdAt: string (nullable = true)
 |-- Exploded_feeds_description: string (nullable = true)
 |-- Exploded_feeds_feedsComment: string (nullable = true)
 |-- Exploded_feeds_id: long (nullable = true)
 |-- Exploded_feeds_imagePaths: string (nullable = true)
 |-- Exploded_feeds_images: string (nullable = true)
 |-- Exploded_feeds_isdeleted: boolean (nullable = true)
 |-- Exploded_feeds_lat: long (nullable = true)
 |-- Exploded_feeds_lng: long (nullable = true)
 |-- Exploded_feeds_location: string (nullable = true)
 |-- Exploded_feeds_mediatype: long (nullable = true)
 |-- Exploded_feeds_msg: string (nullable = true)
 |-- Exploded_feeds_name: string (nullable = true)
 |-- Exploded_feeds_profilePicture: string (nullable = true)
 |-- Exploded_feeds_title: string (nullable = true)
 |-- Exploded_feeds_userId: long (nullable =